# Trading with Python and the Binance API

## API Preparation - Acquire API Key

Navigate to [Binance paper trading](https://testnet.binance.vision/) and login with `github`.
[Generate HMAC_SHA256 Key](https://testnet.binance.vision/key/generate) and save to [binance.properties](../.config/test/binance.properties).

In order to load your Binance credentials:
```
pip install jproperties
```

In [1]:
from jproperties import Properties

properties = Properties()

with open("../.config/test/binance.properties", "rb") as binanceProperties:
  properties.load(binanceProperties)

print(properties.get("api-key"))
print(properties.get("secret-key"))

PropertyTuple(data='T676iQh9CncaNbbkPysmYVXAgJCVXHyZvoInM6dQdhZ85jHhlM1Yb7lI6vEkQDKH', meta={})
PropertyTuple(data='TzYhaKDc8w3WASWn7mGn3XFTVhhOb0lAXpJH8iwLaQAUq4noTeF8XHOsobn0aVfZ', meta={})


## Required packages

Use the unofficial Python wrapper for the Binance exchange REST API version 3.
```
pip install python-binance
```

## Connecting to the API Server

In [10]:
from binance.client import Client
from dataclasses import dataclass
from jproperties import Properties
import os
import pandas as pd

In [3]:
@dataclass(frozen = True)
class BinanceCredentials:
  apiKey: str
  secretKey: str

In [4]:
def binanceCredentials(path):
  with open(path, "rb") as binanceProperties:
    properties = Properties()
    properties.load(binanceProperties)
    return BinanceCredentials(properties.get("api-key").data, properties.get("secret-key").data)

In [5]:
creds = binanceCredentials(os.path.abspath("../.config/test/binance.properties"))

In [6]:
creds

BinanceCredentials(apiKey='T676iQh9CncaNbbkPysmYVXAgJCVXHyZvoInM6dQdhZ85jHhlM1Yb7lI6vEkQDKH', secretKey='TzYhaKDc8w3WASWn7mGn3XFTVhhOb0lAXpJH8iwLaQAUq4noTeF8XHOsobn0aVfZ')

In [7]:
# Paper trading at: https://testnet.binance.vision/
# Is it enough to set testnet = True? Because when we check the URL we get https://api.binance.com/api
client = Client(creds.apiKey, creds.secretKey, tld = "com", testnet = True)

In [8]:
client.API_URL

'https://api.binance.com/api'

In [13]:
account = client.get_account()
account

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1680671854117,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [11]:
# Test connectivity:
client.ping()

{}

In [12]:
client.get_system_status()

{'status': 0, 'msg': 'normal'}

In [14]:
account["accountType"]

'SPOT'

In [15]:
pd.to_datetime(account["updateTime"], unit = "ms")

Timestamp('2023-04-05 05:17:34.117000')

In [16]:
account["balances"]

[{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'},
 {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
 {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
 {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
 {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
 {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
 {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'},
 {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}]

In [17]:
df = pd.DataFrame(account["balances"])
df

,asset,free,locked
0,BNB,1000.00000000,0.00000000
1,BTC,1.00000000,0.00000000
2,BUSD,10000.00000000,0.00000000
3,ETH,100.00000000,0.00000000
4,LTC,500.00000000,0.00000000
5,TRX,500000.00000000,0.00000000
6,USDT,10000.00000000,0.00000000
7,XRP,50000.00000000,0.00000000


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   asset   8 non-null      object
 1   free    8 non-null      object
 2   locked  8 non-null      object
dtypes: object(3)
memory usage: 320.0+ bytes


In [21]:
df.free = pd.to_numeric(df.free, errors = "coerce")
df.locked = pd.to_numeric(df.locked, errors = "coerce")
df

,asset,free,locked
0,BNB,1000.0,0.0
1,BTC,1.0,0.0
2,BUSD,10000.0,0.0
3,ETH,100.0,0.0
4,LTC,500.0,0.0
5,TRX,500000.0,0.0
6,USDT,10000.0,0.0
7,XRP,50000.0,0.0


In [22]:
df.info()0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   asset   8 non-null      object 
 1   free    8 non-null      float64
 2   locked  8 non-null      float64
dtypes: float64(2), object(1)
memory usage: 320.0+ bytes


In [25]:
df.loc[df.free > 1000]

,asset,free,locked
2,BUSD,10000.0,0.0
5,TRX,500000.0,0.0
6,USDT,10000.0,0.0
7,XRP,50000.0,0.0


In [26]:
client.get_asset_balance("BTC")

{'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}

In [28]:
float(client.get_asset_balance("BTC")["free"])

1.0

In [27]:
client.get_asset_balance("ETH")

{'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}